# Drug Production

In [ ]:
import numpy as np
from scipy.io import loadmat
from operator import itemgetter
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
mat = loadmat('cost_vec_data2.mat')
A, b, c_hat, rho, deltaC = itemgetter('A', 'b', 'c', 'rho', 'deltaC')(mat)
b, c_hat, rho = b[0], c_hat[0], rho[0][0]

## Part 1

In [ ]:
def solve_nominal(A, b, c_hat):
    """
    Solves the nominal optimization problem which ignores uncertainty in c.
    
    Returns:
        x_nom: The nominal solution
        cost_nom: The cost of the nominal solution
    """
    x_nom = None
    cost_nom = None
    # TODO: compute x_nom and cost_nom
    return x_nom, cost_nom

In [ ]:
def worst_cost(x, c_hat, rho, deltaC):
    """
    Computes the worst case cost of a decision x.
    
    Returns:
        The worst case cost
    """
    cost_worst = None
    # TODO: compute cost_worst
    return cost_worst

In [ ]:
x_nom, cost_nom = solve_nominal(A, b, c_hat)
print('x_nom:\n{}'.format(x_nom))
print('profit_nom: {}'.format(-1. * cost_nom))
print('worst_profit: {}'.format(-1. * worst_cost(x_nom, c_hat, rho, deltaC)))

## Part 2

In [ ]:
def solve_robust(A, b, c_hat, rho, deltaC):
    """
    Solves the robust optimization problem which considers uncertainty in c.
    
    Returns:
        x_rob: The robust solution
        cost_rob: The cost of the robust solution
    """
    x_rob = None
    cost_rob = None
    # TODO: compute x_rob and cost_rob
    return x_rob, cost_rob

In [ ]:
x_rob, cost_rob = solve_robust(A, b, c_hat, rho, deltaC)
print('x_rob:\n{}'.format(x_rob))
print('profit_rob: {}'.format(-1. * cost_rob))

## Part 3

In [ ]:
rhos = np.linspace(0.03, 0.09, 200)
profits = None
# TODO: compute array of profits corresponding to each rho in rhos

In [ ]:
plt.figure()
plt.title('Profit vs. rho')
plt.plot(rhos, profits)
plt.xlim(0.03, 0.09)
plt.ylim(-1000, 6000)
plt.show()
plt.close()